## Camera Calibration Process:

## 0. Preparation

In [1]:
import cv2
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob 
import matplotlib.pyplot as plt
import os
import json


In [2]:
# Auxiliar functions
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

def write_image(output_folder: str, img_name: str, img: np.array):
    img_path = os.path.join(output_folder, img_name)  
    os.makedirs(output_folder, exist_ok=True)
    cv2.imwrite(img_path, img)  

def show_image(img: np.array, img_name: str = "Image"):
    cv2.imshow(img_name, img)  
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

### Load the chessboard calibration images

In [3]:
imgs_gen_path = 'calibration_images/*.jpeg' 
imgs_path = sorted(glob.glob(imgs_gen_path))
imgs = load_images(imgs_path)  

imgs_path

['calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (1).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (2).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (3).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (4).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (5).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59 (6).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.19.59.jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (1).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (10).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (11).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (12).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (13).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (2).jpeg',
 'calibration_images\\WhatsApp Image 2025-11-30 at 13.20.00 (3).jpeg',
 'cali

### Detect the corners of the pattern and refine them

In [4]:
# Number of rows and cols
pattern_size = (7,7)

In [5]:
corners = [cv2.findChessboardCorners(
    cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), pattern_size) 
    for img in imgs]  

In [6]:
len(corners)  

48

In [7]:
corners[12][1].shape  

(49, 1, 2)

In [8]:
# Security copy
corners_copy = copy.deepcopy(corners)  

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01) 

# To refine corner detections we input grayscale images
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], pattern_size, (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

In [9]:
corners_refined[12].shape


(49, 1, 2)

### Check that the detection process was succesful drawing the detected corners

In [10]:
imgs_copy = copy.deepcopy(imgs) 

In [11]:
output_folder_refined_corners = "refined_corners_output"

for i in range(len(corners_copy)):
    ret = corners_copy[i][0]
    if ret: 
        cv2.drawChessboardCorners(imgs_copy[i], pattern_size, corners_refined[i], ret) 
        #cv2.imshow(f"Img {i} and corners", imgs_copy[i]) 
        write_image(output_folder=output_folder_refined_corners, img_name=f"refined_corners_{i}.jpg", img=imgs_copy[i])
        cv2.waitKey(0)

cv2.destroyAllWindows()

In [12]:
def get_chessboard_points(chessboard_shape, dx, dy):

    cols, rows = chessboard_shape 

    grid_xy = np.mgrid[0:cols, 0:rows].T.reshape(-1,2)

    objp = np.zeros([grid_xy.shape[0], 3], dtype=np.float32) 

    objp[:,0] = grid_xy[:,0]*dx  
    objp[:,1] = grid_xy[:,1]*dy  

    return objp  

In [13]:
# Needs to be changed to the real position of our chessboard corners
dx, dy = 32, 32  
chessboard_points = [get_chessboard_points(pattern_size, dx, dy) for _ in corners_refined]


In [14]:
chessboard_points[0]

array([[  0.,   0.,   0.],
       [ 32.,   0.,   0.],
       [ 64.,   0.,   0.],
       [ 96.,   0.,   0.],
       [128.,   0.,   0.],
       [160.,   0.,   0.],
       [192.,   0.,   0.],
       [  0.,  32.,   0.],
       [ 32.,  32.,   0.],
       [ 64.,  32.,   0.],
       [ 96.,  32.,   0.],
       [128.,  32.,   0.],
       [160.,  32.,   0.],
       [192.,  32.,   0.],
       [  0.,  64.,   0.],
       [ 32.,  64.,   0.],
       [ 64.,  64.,   0.],
       [ 96.,  64.,   0.],
       [128.,  64.,   0.],
       [160.,  64.,   0.],
       [192.,  64.,   0.],
       [  0.,  96.,   0.],
       [ 32.,  96.,   0.],
       [ 64.,  96.,   0.],
       [ 96.,  96.,   0.],
       [128.,  96.,   0.],
       [160.,  96.,   0.],
       [192.,  96.,   0.],
       [  0., 128.,   0.],
       [ 32., 128.,   0.],
       [ 64., 128.,   0.],
       [ 96., 128.,   0.],
       [128., 128.,   0.],
       [160., 128.,   0.],
       [192., 128.,   0.],
       [  0., 160.,   0.],
       [ 32., 160.,   0.],
 

In [15]:
chessboard_points[0].shape

(49, 3)

### Obtain the calibration parameters for our camera

In [16]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]  

# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [17]:
img_size = (imgs_gray[0].shape[1], imgs_gray[0].shape[0]) 

rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points, valid_corners, img_size, None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))  # de esta forma obtenemos los parámetros extrínsecos de la calibración de la cámara

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\calibration.cpp:3397: error: (-2:Unspecified error) in function 'void __cdecl cv::collectCalibrationData(const class cv::_InputArray &,const class cv::_InputArray &,const class cv::_InputArray &,int,class cv::Mat &,class cv::Mat &,class cv::Mat *,class cv::Mat &)'
>  (expected: 'nimages == (int)imagePoints1.total()'), where
>     'nimages' is 48
> must be equal to
>     '(int)imagePoints1.total()' is 43


In [ ]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("\nDistortion coefficients:\n", dist_coeffs)
print("\nRoot mean squared reprojection error:\n", rms)
print("\nExtrinsics:\n", extrinsics)

Intrinsics:
 [[417.77315866   0.         156.33829983]
 [  0.         421.35480199 134.32880063]
 [  0.           0.           1.        ]]

Distortion coefficients:
 [[-2.13049815e-01  2.86830664e+00  5.90990527e-03 -3.86453734e-03
  -2.23696363e+01]]

Root mean squared reprojection error:
 0.1846287136881559

Extrinsics:
 [array([[ 7.14810441e-01,  6.18321143e-01, -3.26687920e-01,
        -3.08750936e+01],
       [-1.82439734e-01,  6.15857353e-01,  7.66443387e-01,
        -3.73702013e+01],
       [ 6.75101309e-01, -4.88260878e-01,  5.53027611e-01,
         5.18490853e+02]]), array([[ 7.19016600e-01,  6.02026625e-01, -3.47244973e-01,
        -5.72888930e+01],
       [-1.91372353e-01,  6.51826460e-01,  7.33824835e-01,
        -4.64598332e+01],
       [ 6.68125550e-01, -4.61179151e-01,  5.83885297e-01,
         5.07766905e+02]]), array([[ 7.28774287e-01,  5.83716857e-01, -3.57998142e-01,
        -6.83872714e+01],
       [-2.03244636e-01,  6.83645704e-01,  7.00942344e-01,
        -6.2118

In [ ]:


def save_calibration_data(mtx, dist, rvecs, tvecs, filename="calibration_data.json"):
    """
    Saves the intrinsic matrix, distortion coefficients, and extrinsic parameters
    (rvecs, tvecs) to a JSON file.
    """
    print("Saving calibration data...")
    
    # Convert NumPy arrays to standard Python lists
    if isinstance(mtx, np.ndarray):
        mtx_list = mtx.tolist()
    else:
        mtx_list = mtx

    if isinstance(dist, np.ndarray):
        # Flatten the distortion coefficients array before converting
        dist_list = np.ravel(dist).tolist() 
    else:
        dist_list = dist

    # Process Extrinsics (rvecs and tvecs are lists of arrays)
    # Convert each array within the list to a Python list
    rvecs_list = []
    if rvecs is not None:
        for r in rvecs:
            if isinstance(r, np.ndarray):
                rvecs_list.append(r.tolist())
            else:
                rvecs_list.append(r)

    tvecs_list = []
    if tvecs is not None:
        for t in tvecs:
            if isinstance(t, np.ndarray):
                tvecs_list.append(t.tolist())
            else:
                tvecs_list.append(t)

    # Create the data dictionary
    data = {
        "intrinsics": mtx_list,
        "distortion": dist_list,
        "extrinsics": {
            "rvecs": rvecs_list,
            "tvecs": tvecs_list
        }
    }

    # Write to file
    try:
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)
        print(f"Success! Data saved to '{filename}'")
        print(f"   Matrix: \n{np.array(mtx_list)}")
        print(f"   Distortion: {dist_list}")
        print(f"   Extrinsics saved: {len(rvecs_list)} vector pairs.")
    except Exception as e:
        print(f"Error saving the file: {e}")

if 'intrinsics' in locals() and 'dist_coeffs' in locals() and 'rvecs' in locals() and 'tvecs' in locals():
    save_calibration_data(intrinsics, dist_coeffs, rvecs, tvecs)
else:
    print("Saving error: Required calibration variables not found in local scope.")